# EP - PSI3471 (2024 - $1^o$ semestre)

|Nome do aluno                     |NUSP    |E-mail USP       |
|----------------------------------|--------|-----------------|
|Gustavo Henrique da Silva Amaral  |12551686|gustavo.amaral7@usp.br|
|Thiago da Rocha Calomino Gonçalves|12554647|thcalomino@usp.br|

Link: [Enunciado do EP 2024](http://www.lps.usp.br/hae/psi3471/ep1-2024/index.html)

# Instalação de dependências

In [2]:
!pip install opendatasets

import sys
import time
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import pandas as pd
import opendatasets as od
import cv2

import tensorflow.keras as keras
from keras.datasets import mnist
from sklearn import tree
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

#import requests
#from PIL import Image
#from io import BytesIO

In [3]:
import cv2
import glob
import os

def reduce_image(image_path, output_size):
    image = cv2.imread(image_path)
    reduced_image = cv2.resize(image, (output_size, output_size), interpolation=cv2.INTER_AREA)
    return reduced_image

def save_image(image, output_path):
    cv2.imwrite(output_path, image)

input_images = glob.glob("path_to_images/*.jpg")
output_size = 100  # Por exemplo, reduzindo para 100x100 pixels

for image_path in input_images:
    reduced_image = reduce_image(image_path, output_size)
    output_path = os.path.join("path_to_output", f"R_{os.path.basename(image_path)}")
    save_image(reduced_image, output_path)


In [4]:
import numpy as np

def findCenterAndOrientation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(binary)
    cen_x = moments['m10'] / moments['m00']
    cen_y = moments['m01'] / moments['m00']
    theta = 0.5 * np.arctan2(2 * moments['mu11'], moments['mu20'] - moments['mu02'])
    return cen_x, cen_y, theta

def align_image(image):
    cen_x, cen_y, theta = findCenterAndOrientation(image)
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, np.degrees(theta), 1.0)
    aligned_image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return aligned_image

for image_path in input_images:
    reduced_image = reduce_image(image_path, output_size)
    aligned_image = align_image(reduced_image)
    output_path = os.path.join("path_to_output", f"A_{os.path.basename(image_path)}")
    save_image(aligned_image, output_path)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def load_images(image_paths):
    images = [cv2.imread(image_path) for image_path in image_paths]
    return np.array([cv2.resize(image, (output_size, output_size)).flatten() for image in images])

kirmizi_images = glob.glob("path_to_output/R_kirmizi*.jpg")
siirt_images = glob.glob("path_to_output/R_siirt*.jpg")

kirmizi_labels = [0] * len(kirmizi_images)
siirt_labels = [1] * len(siirt_images)

all_images = kirmizi_images + siirt_images
all_labels = kirmizi_labels + siirt_labels

X = load_images(all_images)
y = np.array(all_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


ValueError: With n_samples=0, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

def load_images_for_cnn(image_paths):
    images = [cv2.imread(image_path) for image_path in image_paths]
    return np.array([cv2.resize(image, (output_size, output_size)) for image in images])

X = load_images_for_cnn(all_images)
y = to_categorical(all_labels, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(output_size, output_size, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")
